# Data Driven Nadex Binary Options Trading

## A mathematical model to determine entry and exit points for disciplined binary option trading


### Trading Idea

#### Binary option background
Nadex binary options trade between 0 and 100 and pay off 0 if the underlying (the S&P 500 Future or one of several other US or global futures, currency exchange rates, or other indices) is at or below the strike price at expiration. They pay off 100 if the underlying is above the strike.  They trade weekly, daily, 2-hour and 20-minute durations.  To short the index, you sell an option.  There is no put option.  Selling at 30 actually costs 100-30 = \\$70 and gives you a short option with an inverted payoff.  

If you buy at 70 or sell at 30, you invest \\$71 either way. (\\$1 commission on all trades and settlements in the money, losing out of the money doesn't cost a fee).  If you sell (buy) to close that long (short) option at 60 (40), you lose \\$10 plus fees on both ends for -\\$12 net.  If the option finishes in the money you make (lose) 100-70-2 = \\$28  (30-100-1 = -\\$71).

Because binary options always trade between 0 and 100, these instruments trade very differently from stocks or regular options.  We can count on the prices always to be in this range, so we can key on exact, repeatable option price values in ways one could not on a stock or the future or index underlying the binary option. 

In fact binary options always behave in a fairly consistent and similar manner. Their time value decays much more quickly than standard options, because their durations are much shorter.  As the options move through the final half of their lifetimes, they are near 0 (under 20) if the underlying is a strike or two below the strike price and near 100 (above 80) if the UL is well above the strike price.  The option whose strike is nearest the underlying price remains somewhere near 50 until the last few minutes.  If the UL crosses its strike, then the option jumps up and down around 50.  A very risky situation indeed.  If the UL stays on one side or the other of the strike, then the binary option decays from the mid-range to 0 or 100.

#### Trading strategy development

Will trade on the time decay of 20-minute options in the last 10 minutes before expiration.  May also collect data on the last half hour of 2-hour options.  Will buy at a point where partial decay gives a return at least as big as the partial loss exit point.  Will determine exact entry, profit exit and loss exit points based on frequency of outcomes in collected data.  

That's approximately sell at 35, take profit at 10, cut loss at 50, or buy at 65, take profit at 90, cut loss at 50.  These entry and exit points yield a risk of \\$17 versus a reward of \\$23.  If we can do that with discipline (it's hard to catch an option at exactly 50 as they are most dynamic between 40 and 60), then a 50\% probability of win is profitable.

To set the entry and exit points (price, possibly with time range or path dependencies like hysteresis) for an expectation of profitability, I will collect data at the option level to determine approximate probabilities of late-term option finishing in or out of the money and of moving past the 50 mark.  Not going to get involved with full lifetime of the option, even 20-minute options, nor try to play the movement of the underlying index.

### Step 1. Data Collection

Will start with manual data collection and move to web scraping after the concept is proven.

A. Save images of lifetime bid and ask priceline graphs of a few options near the money of a given expiration.  Live Nadex windows easily switch between bid and ask for some time after expiration, but eventually grow stale.  Must capture screen shots for each bid and ask timeline.  Capture UL price also, if possible.  

The market action at Nadex is extremely thin, so last option price is not useful and price improvement seldom works, except in the sense that you can post a bid or offer that may be filled, but usually only when the price moves against you.  Therefore I will work only with bid and ask pricelines.

B.  Collect price points in a Google Sheet.  

#### Price History DataSheet and DataFrame:
UL, Date, Expiration, Strike, PriceTime, TimeToExp, Bid, Ask, Horizon

Bid and Ask can each be a single price represented as a scalar.
If only one side is recorded at a given time then the other side will be NaN.
Or Bid and/or Ask can be lists, in which case the list is the OHLC price range at the given minute, in this order \[Open, High, Low, Close\].

Collect times at which potential entry prices occur.  Store as scalar in Bid or Ask column.
For short strategy:  
1. Bid at 25, 30, 33, 34, 35, 36, 37, 38, 39, 40, 45 in the time range from 9 minutes togo to 60 seconds togo.  
For long strategy:
2. Ask at 75, 70, 67, 66, 65, 64, 63, 62, 61, 60, 55 in the time range from 9 minutes togo to 1 min.

Collect OHLC prices for each minute after the first potential entry on each timeline.  Store as a list in Bid or Ask column.
1. Ask = \[Open, High, Low, Close\] (after first Bid)
2. Bid = \[Open, High, Low, Close\] (after first Ask)

Add an Expiry row at Expiration with Time=Expiration, TimeToExp=0, Price 0 or 100 and "Exp".

Collect expiry price (0 or 100 at 0 TimeToExp) as a scalar in both columns.
1. Bid = 0 or 100, Horizon = Expy, TTX = 0
2. Ask = 0 or 100, Horizon = Expy, TTX = 0

Add a Settled row when the price has "escaped" the side we want it to finish on.  That is, if and when the price goes above 50 to never return to the short side of 50, I'll label an Ask (buy back a short) row with the Horizon column value "Setl".  Similarly for a potential long option that goes below 50 for the final time.

Collect time at which ask (or bid) goes above (below) 50, never to return.
1. Ask = 51, Horizon = Setl, TimeToExp = TTX at last 51
or
1. Bid = 49, Horizon = Setl, TimeToExp = TTX at last 49

C.  Read Sheet via Pandas read_csv, after manually downloading Google Sheet as CSV file in the local Git project Data sub-directory.  Unfortunately, Google overly complicates usage of Google Sheets with their access requirements, since they are a distributed solution.


### Step 2. Trading Model Development / Scenario Data Construction

#### Potential trade entry and exit points

I generally want to find conditional probabilities of reaching certain profit or loss exits or expiration settlements, given that we entered at a certain price with a certain time to expiration (time to go).  From these data, I will select the trading strategy and the entry and exit prices.

Each initial bid or ask recorded is a potential entry point.  Each subsequent ask or bid OHLC contains a set of potential exit points.  In the selling strategy, the ask prices less than the initial bid are profit exits and the ask prices greater than the initial bid are loss exits.  Similarly for the buying strategy with inverted directions.

#### Trading logic Python function

We'll build Python functions to read the Price History data and generate candidate position lifetimes (buy and sell).  I will accumulate these candidate positions in a data frame with profits and losses and returns for each strategy and entry/exit point combination.  

#### Trading strategy

I will try a few different simple trading tactics and compare and combine them to find the best strategy.  Two definitions of best will be considered: 1. Minimize the probability of losses.  2. Maximize the expected return.

I hope to use this data and learning techniques to actually find regions of prices, times to expiration, and other parameters that yield acceptably low likelihood of losses (close to or at 0%).  I'll call that an acceptability region.  Within the Acceptability Region, I hope to find an Optimal Region, where returns are optimized.

##### Buy and Hope

For stocks the popular buy and hold strategy would be better called buy and hope.  In any kind of options, it is a very naive strategy indeed, but working the time decay near expiration it may give us spectacular gains more often than catastrophic losses.  There are only two parameters with Buy and Hold, the entry price and entry time, as the exit point/time is just expiration.

##### Fixed Entry, Profit Exit and Loss Exits

Enter positions at one price level and exit at one profit point or one loss point.  Find best combination of each price level.

##### Fixed Entry, Fixed Relative Exit and Loss Exits

Enter positions at one point and exit at one profit point or loss point that are defined as offsets relative to the entry point.

##### Hysteresis Profit Exit, Fixed Loss Exit

One entry price, one loss exit and take profit after price moves favorably and then moves against us a certain amount.

##### Hysteresis Exit Points

One entry price, hysteresis profit and exit prices.

##### Price Pattern Signals

I will look at price patterns, referred to as technical indicators, as a possible tactic.  However this would likely be another large effort on its own, so if it is used, it will be a very minimal approach to aid the basic price/time/hysteresis tactics.

#### Scenario Data Construction

The Trading model will find all the candidate entries and exits for each tactic within the available price data.  Further code will store those in a data frame along with profits/losses and returns.

This candidate position data will be our basis for Machine Learning/Data Mining to select an exact trading strategy or ensemble of strategies.  I will look for trading parameters that give us predominantly profits using classifcation.  I also hope to look for parameters to optimize, or at least improve, returns using regression or optimization.

A third opportunity to possibly use machine learning is at run time.  If the trading logic turns out fairly complicated, I may use classification to generate buy and sell signals based on price movements in real time.




### Step 3. EDA

### Step 4.  Model Logic

### Step 5.  Model Statistics

### Step 6.  Trading Model